In [ ]:
from bmtk.builder.networks import NetworkBuilder
from bmtk.utils.sim_setup import build_env_bionet
import json
from edit_genome import process_json
from bmtk.simulator import bionet
from bmtk.analyzer.spike_trains import to_dataframe
from bmtk.analyzer.cell_vars import plot_report
import os
import shutil
from neuron import h
import ipywidgets as widgets
from ipywidgets import HBox,VBox,Label,Layout
from IPython.display import display
%matplotlib inline

In [ ]:
a = widgets.FloatText(description="Rin(Mohm)", continuous_update=False)
b = widgets.FloatText(description="Tau(ms)", continuous_update=False)
c = widgets.FloatText(description="Vrest(mV)", continuous_update=False)
ui1 = HBox([VBox([Label('Passive')],layout=Layout(width='10%')),VBox([a,b,c],layout=Layout(width='38%'))])
ui1

In [ ]:
w_0 = widgets.Checkbox(value=False,description="Injected Current",disabled=False)
text1 = widgets.Text()
text2 = widgets.Text()
amplitude = []
frequency = []
def update_0(*arg):
    if w_0.value:
        text1.description='Amplitude:'
        def amplitude_value(sender):
            for i in sender.value.split(','):
                amplitude.append(i)
            
            print('amplitude_value',amplitude,'spiking spots number:',len(amplitude))
            return
        text1.on_submit(amplitude_value)
        text2.description='Frequency:'
        def frequency_value(sender):
            for i in sender.value.split(','):
                frequency.append(i)
        
            print('frequency_value',frequency,'spiking spots number:',len(frequency))
            return
        text2.on_submit(frequency_value)
        display(text1,text2)
w_0.observe(update_0,'value')
w_0

In [ ]:
gids = []
amplitude_run = []
for i in amplitude:
    amplitude_run.append(i)
print(amplitude_run)
amplitude_run.insert(0,'-0.1')
for i in range (len(amplitude_run)):
    gids.append(i)
amplitude.insert(0,'-0.1')
frequency.insert(0,'0')
print(amplitude,frequency)

In [ ]:
from bmtk.builder.networks import NetworkBuilder

net = NetworkBuilder('BL')
net.add_nodes(N=len(amplitude)+1,cell_name='BL',
              potental='exc',
              model_type='biophysical',
              model_template='ctdb:Biophys1.hoc',
              morphology='reconstruction.swc',
              model_processing='aibs_allactive',
              dynamics_params='BL_fit.json'
              )
shutil.rmtree('./network')
net.build()
net.save_nodes(output_dir='network')

from bmtk.utils.sim_setup import build_env_bionet

build_env_bionet(base_dir='PN_IClamp',      # Where to save the scripts and config files 
                 components_dir='components',
                 network_dir='network',    # Location of directory containing network files
                 tstop=2000.0, dt=0.1,     # Run a simulation for 2000 ms at 0.1 ms intervals
                 report_vars=['v'], # Tells simulator we want to record membrane potential and calcium traces
                 current_clamp={           # Creates a step current from 500.ms to 1500.0 ms  
                     'amp': amplitude_run ,
                     'delay': 500.0,
                     'duration': 1000.0,
                     'gids': gids
                 }
                )
shutil.copy('sources/BL_fit.json','components/biophysical_neuron_models/')
shutil.copy('sources/neuronal_model_491766131/reconstruction.swc','components/morphologies/')
shutil.rmtree('components/mechanisms/')
shutil.copytree('sources/modfiles','components/mechanisms/modfiles')
shutil.copytree('sources/x86_64','components/mechanisms/x86_64')

In [ ]:
filePath = './sources/modfiles'
a = os.listdir(filePath)

w_0 = widgets.Checkbox(value=False,description=a[2],disabled=False)
w_1 = widgets.Checkbox(value=False,description=a[18],disabled=False)
w_2 = widgets.Checkbox(value=False,description=a[12],disabled=False)

def update_0(*args):
    if w_0.value:
        text0 = widgets.FloatText(
            value=7.5,
            description='NaP:',
            disabled=False
        )
        value0 = text0.value*1e-6
        process_json('./components/biophysical_neuron_models/BL_fit.json','Nap',value0)
        display(text0)
def update_1(*args):
    if w_1.value:
        text1 = widgets.FloatText(
            value=7.5,
            description='Ih:',
            disabled=False
        )
        value1 = text1.value*1e-6
        process_json('./components/biophysical_neuron_models/BL_fit.json','Ih',value1)
        display(text1)        
def update_2(*args):
    if w_2.value:
        text2 = widgets.FloatText(
            value=7.5,
            description='Im:',
            disabled=False
        )
        value2 = text2.value*1e-3
        process_json('./components/biophysical_neuron_models/BL_fit.json','Im',value2)
        display(text2)              
        
w_0.observe(update_0,'value')
w_1.observe(update_1,'value')
w_2.observe(update_2,'value')
ui1 = HBox([VBox([Label('Channels')],layout=Layout(width='10%')),VBox([w_0,w_1,w_2],layout=Layout(width='38%'))])
ui1


In [ ]:
from bmtk.simulator import bionet
conf = bionet.Config.from_json('PN_IClamp/simulation_config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
pc = h.ParallelContext()
hcell = pc.gid2cell(0)
#inf  = h.Vector()
#inf.record(hcell.soma[0]._ref_ninf_im)

w_3 = widgets.Checkbox(value=False,description=a[2],disabled=False)
w_5 = widgets.Checkbox(value=False,description=a[12],disabled=False)
def update_3(*args):
    if w_3.value:
        FloatText1 = widgets.FloatText(
            value=-38.43,
            description='Vhalf_m:',
            disabled=False
        )
        hcell.soma[0].mhalf_Nap = FloatText1.value
        FloatText4 = widgets.FloatText(
            value=7.2,
            description='K_m:',
            disabled=False
        )
        hcell.soma[0].slope1_Nap = FloatText4.value
        display(FloatText1,FloatText4)
                
def update_5(*args):
    if w_5.value:       
        FloatText2 = widgets.FloatText(
            value=-52.7,
            description='Vhalf_n:',
            disabled=False
        )
        hcell.soma[0].vhalf_im  = FloatText2.value
        FloatText5 = widgets.FloatText(
            value=10.34,
            description='K_n:',
            disabled=False
        )
        hcell.soma[0].k_im = FloatText5.value
        display(FloatText2,FloatText5)
w_3.observe(update_3,'value')
w_5.observe(update_5,'value')
            
ui2 = HBox([VBox([Label('Vhalf&slope')],layout=Layout(width='10%')),VBox([w_3,w_5],layout=Layout(width='38%'))])
ui2


In [ ]:
sim.run()

In [ ]:
from bmtk.analyzer import nodes_table
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pdb
from mpl_toolkits.mplot3d import Axes3D
import h5py
from bmtk.analyzer.cell_vars import _get_cell_report, plot_report
import matplotlib.pyplot as plt
from scipy.signal import welch

# Load data
config_file = "simulation_config.json"
#lfp_file = "./output/ecp.h5"
mem_pot_file = './PN_IClamp/output/v_report.h5'
raster_file = './PN_IClamp/output/spikes.h5'


# load 
f = h5py.File(mem_pot_file,'r')
F =h5py.File(raster_file,'r')
mem_potential = f['report']['BL']['data']
gids = F['spikes']['BL']['node_ids']
timestamps = F['spikes']['BL']['timestamps']

In [ ]:
duration = 1000  #ms
spikesnum = []
for i in range (len(amplitude_run)):
    count = 0
    for elements in gids.value:
        if elements == i:
            count +=1
    spikesnum.append(count)
frequency1 = []
for elements in spikesnum:
    freq = elements*1000/duration
    frequency1.append(freq)

In [ ]:
plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
plt.plot(f['report']['BL']['data'][:,0],label = 'Passive')
plt.legend()
plt.subplot(2,2,2)

for i in range(0, len(frequency)): 
    frequency[i] = float(frequency[i]) 
for i in range(0, len(amplitude)): 
    amplitude[i] = float(amplitude[i]) 
plt.plot(amplitude_run,frequency,'yo',label = "User's FIR")
plt.plot(amplitude_run,frequency1,'bo',label = "FIR")
plt.legend()
plt.show()